In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:13 of 0:12:14 remaining]
Chain 1:  10% [0:00:56 of 0:01:02 remaining]
Chain 1:  20% [0:00:55 of 0:01:09 remaining]
Chain 1:  30% [0:00:49 of 0:01:10 remaining]
Chain 1:  40% [0:00:41 of 0:01:09 remaining]
Chain 1:  50% [0:00:34 of 0:01:08 remaining]
Chain 1:  60% [0:00:27 of 0:01:08 remaining]
Chain 1:  70% [0:00:21 of 0:01:10 remaining]
Chain 1:  80% [0:00:14 of 0:01:10 remaining]
Chain 1:  90% [0:00:07 of 0:01:10 remaining]
Chain 1: 100% [0:00:00 of 0:01:10 remaining]

Chain 2:   0% [0:00:04 of 0:00:04 remaining]
Chain 2:  10% [0:00:45 of 0:00:51 remaining]
Chain 2:  20% [0:00:50 of 0:01:03 remaining]
Chain 2:  30% [0:00:48 of 0:01:09 remaining]
Chain 2:  40% [0:00:42 of 0:01:10 remaining]
Chain 2:  50% [0:00:36 of 0:01:13 remaining]
Chain 2:  60% [0:00:29 of 0:01:13 remaining]
Chain 2:  70% [0:00:22 of 0:01:14 remaining]
Chain 2:  80% [0:00:15 of 0:01:15 remaining]
Chain 2:  90% [0:00:08 of 0:01:16 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[23.6039 113.089 0.925014; 27.2215 114.778 0.890756; … ; 25.5551 113.612 0.90938; 25.5023 114.239 0.895432]

[27.3824 117.272 0.832217; 23.6836 117.175 0.83062; … ; 27.7842 116.973 0.824652; 24.5912 115.55 0.855789]

[25.7432 112.332 0.933039; 25.4741 113.642 0.915453; … ; 26.4728 111.841 0.952798; 26.1371 116.775 0.837422]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.1584135 2.001203952 0.01217893941 0.0204036238 9000.0000
beta[1] 113.8669706 1.903653490 0.01158526620 0.0344287740 3057.2622
beta[2]   0.9053412 0.041929642 0.00025517567 0.0007528169 3102.1558

Quantiles:
            2.5%       25.0%        50.0%       75.0%        97.5%   
     s2  22.5643361  24.7672865  26.05200926  27.4498802  30.35740229
beta[1] 110.1938281 112.5601530 113.84147314 115.1686319 117.63770252
beta[2]   0.8231997   0.8769393   0.90557532   0.9339819   0.98608715



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*